In [79]:
# Import library
import numpy as np
import sys
import os
import re
import shutil
import copy

In [80]:
# /* Constant */
AUTHENTIC_ORDER = 0
REVERSE_ORDER = 1

# Yukkuri Type
ORIGINAL            = "元祖　　　　: Original"
KITUNE              = "きつね式　　: Kitune Type"
NEW_KITUNE          = "新きつね式　: New Kitune Type"
FULL_BODY_KITUNE    = "全身きつね式: Full body Kitune Type"
KAMUI               = "神威式　　　: Kamui Type"
RAKUGAKI            = "らくがき式　: Rakugaki Type"
WATASHI             = "私式。　　　: Watashi Type"
PUNI                = "ぷに式　　　: Puni Type"
QH                  = "QH式　　　　: QH Type"
NININON             = "弐ヶのん式　: Nininon Type"
TORIMISO            = "とりみそ式　: ToriMiso Type"
MIKAN               = "みかん式　　: Mikan Type"
KURO                = "クロ式　　　: Kuro Type"
HACHI               = "蜂旧式　　　: Hachi Type"

# Vector [eye, mouth]
# [a: close, b, c: open] -> 0: 正順: 口
# [a: open, b, c: close] -> 1: 逆順: 目
ORIGINAL_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KITUNE_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
NEW_KITUNE_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8) # special
FULL_BODY_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KAMUI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
RAKUGAKI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
WATASHI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
PUNI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
QH_VECTOR = np.array([
    REVERSE_ORDER, AUTHENTIC_ORDER, 
    REVERSE_ORDER, REVERSE_ORDER, 
    REVERSE_ORDER
], dtype=np.uint8) # special
NININON_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
TORIMISO_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
MIKAN_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
KURO_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)
HACHI_VECTOR = np.array([REVERSE_ORDER, AUTHENTIC_ORDER], dtype=np.uint8)

# List
FOLDER_LIST = np.array(["目", "口"], dtype=object)
FOLDER_LIST_NEW_KITUNE = np.array(
    [os.path.join(name, num) for name in FOLDER_LIST for num in ["00", "01"]], dtype=object
)
FOLDER_LIST_QH = np.array(["目", "口", "目-どんぐり目", "目-丸目", "目-瞳のある目"], dtype=object)
Y_TYPE_LIST = np.array([
    ORIGINAL,
    KITUNE,
    NEW_KITUNE,
    FULL_BODY_KITUNE,
    KAMUI,
    RAKUGAKI,
    WATASHI,
    PUNI,
    QH, 
    NININON, 
    TORIMISO, 
    MIKAN, 
    KURO, 
    HACHI
], dtype=object)

# Dictionary
Y_VECTOR_DICT = {
    KAMUI: KAMUI_VECTOR,
    RAKUGAKI: RAKUGAKI_VECTOR,
    WATASHI: WATASHI_VECTOR, 
    NEW_KITUNE: NEW_KITUNE_VECTOR, 
    PUNI: PUNI_VECTOR,
    KITUNE: KITUNE_VECTOR,
    FULL_BODY_KITUNE: FULL_BODY_KITUNE,
    ORIGINAL:ORIGINAL_VECTOR, 
    QH: QH_VECTOR, 
    NININON: NININON_VECTOR, 
    TORIMISO: TORIMISO_VECTOR, 
    MIKAN: MIKAN_VECTOR, 
    KURO: KURO_VECTOR, 
    HACHI: HACHI_VECTOR
}

In [93]:
# /* Class */
class Converter:
    def __init__(self, path, name, vector_arr, type):
        self.type = type
        self.path = path
        self.name = name
        self.vector_arr = vector_arr
    
    def folder_copy(self):
        if self.type == NEW_KITUNE:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST_NEW_KITUNE
            )
        elif self.type == QH:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST_QH
            )
        else:
            self.folder_copy_main(
                path=self.path, name=self.path, 
                folder_list=FOLDER_LIST
            )
    
    def folder_copy_main(self, path, name, folder_list):
        folder_path = os.path.join(path, name)
        new_folder_path = os.path.join(path, name+"-YMM4")
        if os.path.isdir(new_folder_path):
            shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
        shutil.copytree(
            folder_path, new_folder_path, 
            ignore=shutil.ignore_patterns(*folder_list), dirs_exist_ok=True
        )
    
    def convert(self):
        print("改名を開始します．\nStart renaming.")
        if self.type == NEW_KITUNE:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST_NEW_KITUNE
            )
        elif self.type == QH:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST_QH
            )
        else:
            self.convert_name(
                path=self.path, name=self.name, 
                vector_arr=self.vector_arr, folder_list=FOLDER_LIST
            )
        print("改名完了\nRenaming completed.")
    
    def convert_name(self, path, name, vector_arr, folder_list):
        original_name = copy.copy(name)
        for folder_name, vector in zip(folder_list, vector_arr):
            target_path = os.path.join(path, folder_name)
            new_folder_path = target_path.replace(original_name, original_name+"-YMM4")
            # 改変後のフォルダー作成
            if os.path.isdir(new_folder_path):
                shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
            elif not os.path.isdir(new_folder_path):
                os.makedirs(new_folder_path) # ディレクトリの作成
            # ファイル名読込
            files = os.listdir(target_path)
            files = np.array(files, dtype=object)
            # 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
            num_arr = np.array([], dtype=object)
            for file in files:
                num_str = re.sub(r"\D", "", file)
                num_arr = np.append(num_str, num_arr)
            num_arr = np.unique(num_arr) # 重複した要素の削除
        # リネーム
            for target_num in num_arr:
                # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
                convert_list = [ s for s in files if re.sub(r"\D", "", s)==target_num ]
                convert_arr = np.array(convert_list, dtype=object)
                # 順番入れ替え
                if vector == AUTHENTIC_ORDER:
                    convert_arr = np.hstack( (convert_arr[-1], convert_arr[:-1]) )
                elif vector == REVERSE_ORDER:
                    c_arr = np.flip(convert_arr[1:])
                    convert_arr = np.hstack( (convert_arr[0], c_arr) )
                for i, file in enumerate(convert_arr):
                    ext = os.path.splitext(file)[1] # 拡張子抽出
                    before_path = os.path.join(target_path, file)
                    if i == 0:
                        rename = target_num+ext # 改変後の名前
                        after_path = os.path.join(new_folder_path, rename)
                    elif i != 0:
                        rename = target_num+"."+str(i-1)+ext # 改変後の名前
                        after_path = os.path.join(new_folder_path, rename)
                    shutil.copy2(before_path, after_path) # Copy file

In [94]:
# /* Function */
# Do not use.
def get_dir_path(relative_path):
    try:
        base_path = sys._MEIPASS
        #print("[Base Path (get from sys)]" + base_path)
    except Exception:
        base_path = os.path.dirname("__file__")
        #print("[Base Path (get from sys)]" + base_path)
    return os.path.join(base_path, relative_path)

def get_folder_name(path):
    files = os.listdir(path)
    files_dir = [f for f in files if os.path.isdir(os.path.join(path, f))]
    files_dir = np.array(files_dir, dtype=object)
    return files_dir

def select_folder(folders):
    print("YMM4 に対応させるフォルダ名を選んでください．")
    print("Select the folder name that corresponds to YMM4.")
    for i in range(len(folders)):
        print("0"+str(i)+": "+folders[i])
    folder_str = input("\n数字を入力して下さい．\nPlease enter a number.\nEx. 1 or 01: ")
    is_number(string=folder_str)
    folder_n = int(folder_str)
    is_corresponding_number(target=folder_n, search_arr=folders)
    print("["+folders[folder_n]+"] を選択しました．\n")
    
    return folders[folder_n]

def is_number(string):
    if( not string.isdigit() ):
        print("入力に文字が検出されました．")
        print("数字のみを入力してください．")
        print("A character was detected in the input.")
        print("Please enter numbers only.")
        
        input("\nPress any key to exit.")
        
        sys.exit()

def is_corresponding_number(target, search_arr):
    if ( not target in range(len(search_arr))):
        print("対応されていない数字が入力されました．")
        print("対応する数字を入力してください．")
        print("An unsupported number has been entered.")
        print("Please enter the corresponding number.")
        
        input("\nPress any key to exit.")
        
        sys.exit()

# Do not use.
def select_direction():
    print("目や口が開いている方向を選んでください．")
    print("Choose the direction in which the eyes and mouth are open.")
    print("\nEx. [a: close, b, c: open] -> 0: 正順")
    print("    [a: open, b, c: close] -> 1: 逆順\n")
    # 目
    eye_n = input("目の数字を入力して下さい．")
    is_number(string=eye_n)
    eye_n = int(eye_n)
    is_corresponding_number(target=eye_n, search_arr=np.arange(2))
    if eye_n == AUTHENTIC_ORDER:
        print("目は [0: 正順] で改名します．")
    elif eye_n == REVERSE_ORDER:
        print("目は [1: 逆順] で改名します．")
    # 口
    mouth_n = input("口の数字を入力して下さい．")
    is_number(string=mouth_n)
    mouth_n = int(mouth_n)
    is_corresponding_number(target=mouth_n, search_arr=np.arange(2))
    if mouth_n == AUTHENTIC_ORDER:
        print("口は [0: 正順] で改名します．")
    elif mouth_n == REVERSE_ORDER:
        print("口は [1: 逆順] で改名します．")
    
    dir_arr = np.array([eye_n, mouth_n], dtype=uint8)
    
    return dir_arr

def convert_name(path, name, vector_arr):
    print("改名を開始します．\nStart renaming.")
    original_name = copy.copy(name)
    for folder_name, vector in zip(FOLDER_LIST, vector_arr):
        target_path = os.path.join(path, folder_name)
        new_folder_path = target_path.replace(original_name, original_name+"-YMM4")
        print("original: "+original_name)
        print(new_folder_path)
        # 改変後のフォルダー作成
        if os.path.isdir(new_folder_path):
            shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
        elif not os.path.isdir(new_folder_path):
            os.mkdir(new_folder_path) # ディレクトリの作成
        # ファイル名読込
        files = os.listdir(target_path)
        #files_file = [f for f in files if os.path.isfile(os.path.join(path, f))]
        files = np.array(files, dtype=object)
        #print(files)
        # 重複ナンバー抽出　例：["01a", "01b", "02a", "02b"] -> ["01", "02"]
        num_arr = np.array([], dtype=object)
        for file in files:
            num_str = re.sub(r"\D", "", file)
            num_arr = np.append(num_str, num_arr)
        num_arr = np.unique(num_arr) # 重複した要素の削除
        #print(num_arr)
        # リネーム
        for target_num in num_arr:
            # EX. ["01a", "01b", "02a", "02b"] -> ["01a", "01b"]
            convert_list = [ s for s in files if re.sub(r"\D", "", s)==target_num ]
            convert_arr = np.array(convert_list, dtype=object)
            # 単一のファイルならブレイク
            #if len(convert_arr) == 1:
            #    break
            # 順番入れ替え
            if vector == AUTHENTIC_ORDER:
                convert_arr = np.hstack( (convert_arr[-1], convert_arr[:-1]) )
            elif vector == REVERSE_ORDER:
                c_arr = np.flip(convert_arr[1:])
                convert_arr = np.hstack( (convert_arr[0], c_arr) )
            for i, file in enumerate(convert_arr):
                ext = os.path.splitext(file)[1] # 拡張子抽出
                before_path = os.path.join(target_path, file)
                if i == 0:
                    rename = target_num+ext # 改変後の名前
                    after_path = os.path.join(new_folder_path, rename)
                elif i != 0:
                    rename = target_num+"."+str(i-1)+ext # 改変後の名前
                    after_path = os.path.join(new_folder_path, rename)
                
                #print("before: "+before_path)
                #print("after : "+after_path)
                #os.rename(before_path, after_path)
                shutil.copy2(before_path, after_path)
    print("改名完了\nRenaming completed.")

def select_type():
    print("改名するゆっくりのタイプを選択してください．")
    print("Select the Yukkuri type of renaming.\n")
    for i, yukkuri in enumerate(Y_TYPE_LIST):
        print("0"+str(i)+": "+yukkuri)
    y_str = input("\n数字を入力して下さい．\nPlease enter a number.\nEx. 1 or 01: ")
    is_number(string=y_str)
    y_num = int(y_str)
    is_corresponding_number(target=y_num, search_arr=Y_TYPE_LIST)
    type_result = Y_TYPE_LIST[y_num].replace("　", "")
    print("["+type_result+"] を選択しました．\n")
    vector = Y_VECTOR_DICT[Y_TYPE_LIST[y_num]]
    return (vector, Y_TYPE_LIST[y_num])

# Do not use.
def folder_copy(path, name):
    folder_path = os.path.join(path, name)
    new_folder_path = os.path.join(path, name+"-YMM4")
    if os.path.isdir(new_folder_path):
        shutil.rmtree(new_folder_path) # ディレクトリを中身ごと削除
    shutil.copytree(
        folder_path, new_folder_path, 
        ignore=shutil.ignore_patterns(FOLDER_LIST[0], FOLDER_LIST[1]), dirs_exist_ok=True
    )

def main():
    # 現在のディレクトリ取得
    cur_path = os.getcwd()
    print("Current directory: "+cur_path+"\n")
    # 改名するフォルダの選択
    folders = get_folder_name(path=cur_path)
    folder_name = select_folder(folders=folders)
    folder_path = os.path.join(cur_path, folder_name)
    # ゆっくりのタイプ選択
    vector_arr, y_type = select_type()
    # 改名処理
    #dir_arr = select_direction()
    converter = Converter(type=y_type, path=folder_path, name=folder_name, vector_arr=vector_arr)
    converter.folder_copy()
    #folder_copy(path=cur_path, name=folder_name)
    #convert_name(path=folder_path, name=folder_name, vector_arr=vector_arr)
    converter.convert()

In [95]:
if __name__ == '__main__':
    main()

Current directory: d:\GitHub-Home\00-JupyterHome\2021\Y3toY4-name-converter

YMM4 に対応させるフォルダ名を選んでください．
Select the folder name that corresponds to YMM4.
00: QH式博麗霊夢
01: Reference
02: Resource
03: きつね式ずんこさん
04: きつね式せいじゃ
05: きつね式せいじゃ-YMM4
06: とりみそ式霊夢
07: ぷにちるの
08: みかん式ゆっくりアリス
09: ゆっくり霊夢改
010: らくがき式霊夢
011: クロ式結月ゆかり
012: 弐ヶのん式霊夢
013: 新きつね式ありす
014: 旧作霊夢
015: 神威式霊夢
016: 私式。霊夢
[きつね式せいじゃ] を選択しました．

改名するゆっくりのタイプを選択してください．
Select the Yukkuri type of renaming.

00: 元祖　　　　: Original
01: きつね式　　: Kitune Type
02: 新きつね式　: New Kitune Type
03: 全身きつね式: Full body Kitune Type
04: 神威式　　　: Kamui Type
05: らくがき式　: Rakugaki Type
06: 私式。　　　: Watashi Type
07: ぷに式　　　: Puni Type
08: QH式　　　　: QH Type
09: 弐ヶのん式　: Nininon Type
010: とりみそ式　: ToriMiso Type
011: みかん式　　: Mikan Type
012: クロ式　　　: Kuro Type
013: 蜂旧式　　　: Hachi Type
[きつね式: Kitune Type] を選択しました．

改名を開始します．
Start renaming.
改名完了
Renaming completed.


In [18]:
input("数字を入力して下さい　Ex. 1 or 01")

'01'

In [23]:
np.arange(2)

array([0, 1])

In [41]:
sample_text = np.array(["01a.png", "01b.png", "01c.png", "02.png"], dtype=np.object)

regex = re.compile('d+')

for line in sample_text.splitlines():
  match = regex.findall(line)
  print(match)

AttributeError: 'numpy.ndarray' object has no attribute 'splitlines'

In [57]:
num_arr = np.array(["01a.png", "01b.png", "01c.png", "02.png"], dtype=np.object)
#[ s for s in num_arr if re.sub(r"\D", "", s)=="01" ]
num_arr = np.flip(num_arr)
num_arr

array(['02.png', '01c.png', '01b.png', '01a.png'], dtype=object)

In [60]:
os.path.splitext(num_arr[0])[1]

'.png'

In [73]:
np.arange(2)

array([0, 1])

In [78]:
num_arr[:-1]

array(['02.png', '01c.png', '01b.png'], dtype=object)

In [79]:
np.hstack((num_arr[-1], num_arr[:-1]))

array(['01a.png', '02.png', '01c.png', '01b.png'], dtype=object)

In [107]:
a = "07a-15"
re.sub(r"\D{2}", "", a)

'0715'

In [114]:
os.listdir(r"D:\GitHub-Home\00-JupyterHome\2021\Y3toY4-name-converter\きつね式せいじゃ\目")

['00.png',
 '00a.png',
 '00b.png',
 '00c.png',
 '00d.png',
 '00e.png',
 '01.png',
 '01a.png',
 '01b.png',
 '01c.png',
 '01d.png',
 '01e.png',
 '02.png',
 '02a.png',
 '02b.png',
 '02c.png',
 '02d.png',
 '02e.png',
 '03.png',
 '03a.png',
 '03b.png',
 '03c.png',
 '03d.png',
 '03e.png',
 '04.png',
 '04a.png',
 '04b.png',
 '04c.png',
 '04d.png',
 '04e.png',
 '05.png',
 '05a.png',
 '05b.png',
 '05c.png',
 '05d.png',
 '05e.png',
 '06.png',
 '06a.png',
 '06b.png',
 '06c.png',
 '06d.png',
 '06e.png',
 '07-15.png',
 '07a-15.png',
 '07b-15.png',
 '07c-15.png',
 '07d-15.png',
 '07e-15.png',
 '08-15.png',
 '08a-15.png',
 '08b-15.png',
 '08c-15.png',
 '08d-15.png',
 '08e-15.png',
 '09-15.png',
 '09a-15.png',
 '09b-15.png',
 '09c-15.png',
 '09d-15.png',
 '09e-15.png',
 '10.png',
 '10k.png',
 '10l.png',
 '10m.png',
 '10n.png',
 '10o.png',
 '11.png',
 '11k.png',
 '11l.png',
 '11m.png',
 '11n.png',
 '11o.png',
 '12.png',
 '12k.png',
 '12l.png',
 '12m.png',
 '12n.png',
 '12o.png',
 '13.png',
 '13k.png',


In [129]:
"[きつね式　　：Kitune Type]".replace("　", "")

'[きつね式：Kitune Type]'

In [17]:
folder_copy(path=os.getcwd(), name="aaa")

In [5]:
python --version

NameError: name 'python' is not defined

In [11]:
FOLDER_LIST.tolist()

['目', '口']

In [53]:
a = np.arange(4)
a

array([0, 1, 2, 3])

In [54]:
a[1:]

array([1, 2, 3])

In [70]:
os.path.join("a", "ppp")

'a\\ppp'

In [76]:
[os.path.join(p_up, p_down) for p_up in ["目", "口"] for p_down in ["aaa", "bbb"]]

['目\\aaa', '目\\bbb', '口\\aaa', '口\\bbb']

In [77]:
aa, bb = (0, 1)

In [78]:
print(aa, bb)

0 1


In [98]:
*[os.path.join(p_up, p_down) for p_up in ["目", "口"] for p_down in ["aaa", "bbb"]]

SyntaxError: can't use starred expression here (Temp/ipykernel_8612/2687076090.py, line 1)